In [1]:
import pandas as pd
import numpy as np

In [2]:
tarihler=pd.read_csv("data_date.txt")
etiketler=pd.read_csv("data_etiket.txt")
haberler=pd.read_csv("data_haber.txt",sep="\n")

In [3]:
veriler=pd.concat([tarihler,haberler,etiketler],axis=1)

In [4]:
veriler

,date,HABER,DURUM
0,20.06.2022,Investing.com - Kripto para piyasalarındaki gü...,artar
1,18.06.2022,Bitcoin 20.000 doların altına düştüInvesting.c...,artar
2,17.06.2022,Investing.com - Kripto para piyasalarındaki gü...,azalir
3,16.06.2022,Investing.com - Kripto para piyasalarındaki gü...,artar
4,15.06.2022,Investing.com - Kripto para piyasalarındaki gü...,azalir
...,...,...,...
1202,30.09.2018,Merkezi olmayan borsaya olan talep yüksektir. ...,azalir
1203,28.09.2018,Investing.com – Kripto paralar Perşembe günü k...,azalir
1204,27.09.2018,"Investing.com - Litecoin Perşembe günü %10,28 ...",azalir
1205,26.09.2018,İngiltere menşeli The Knowledge Academy kurulu...,artar


In [5]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
import jpype
from nltk.stem.porter import PorterStemmer


class Binary:
    def __init__(self):
        ZEMBEREK_PATH = 'zemberek-full_old.jar'
        startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))
        TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
        self.morphology = TurkishMorphology.createWithDefaults()
        nltk.download('stopwords')
        nltk.download('punkt')
    def kelime_temizlik(self,metin):
        yeniMetin=''
        for i in metin:
            if i not in string.punctuation:
                yeniMetin += i
        yorum=yeniMetin
        yorum=yorum.replace("0","").replace("1","").replace("2","").replace("3","").replace("4","").replace("5","").replace("6","").replace("7","").replace("8","").replace("9","")
        yorum=yorum.lower()
        yorum=yorum.split()
        ps=PorterStemmer()
        yorum=[ps.stem(kelime) for kelime in yorum if not kelime in set(stopwords.words('turkish'))]
        yorum= ' '.join(yorum)
        token = word_tokenize(yorum)
        return token

    def kok_temizlik(self,metin):
        token=self.kelime_temizlik(metin)
        lemma_word_top=''
        for i in range(len(token)):
            lemma_word = str(self.morphology.analyzeAndDisambiguate(str(token[i])).bestAnalysis()[0].getLemmas()[0])
            lemma_word_top=f'{lemma_word_top} {lemma_word}'
        file = open("ozel_kelimeler.txt","r",encoding="utf-8")
        kelimeler=file.read().split("\n")
        binary=[]
        for i in range(len(kelimeler)):
            #if metin.count(kelimeler[i])>1:
            #    print(kelimeler[i])
            binary.append(metin.count(kelimeler[i]))
        return binary


In [6]:
bn=Binary()
X=[]
for i in range (len(veriler)):
    X.append(bn.kok_temizlik(veriler.HABER[i]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zetze\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zetze\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
btc=pd.read_csv("btc.csv")

In [8]:
for i in range(len(btc["Şimdi"])): btc["Şimdi"][i] = btc["Şimdi"][i].replace(".","").replace(",",".")

In [9]:
sonraki_kapanislar=[]
for i in range(len(btc["Şimdi"])):
    if i == 0:
        sonraki_kapanislar.append("20720.4")
    else:
        sonraki_kapanislar.append(btc["Şimdi"][i-1])

In [10]:
sonraki_kapanislar= np.asarray(sonraki_kapanislar).astype(np.float32)

In [11]:
hacimler=[]
kapanislar=[]
sonraki_kapanislar_=[]
for i in range (len(btc.Tarih)):
    for j in range(len(veriler.date)):
        if btc.Tarih[i]==veriler.date[j]:
            sonraki_kapanislar_.append(sonraki_kapanislar[i])
            hacimler.append(btc["Hac."][i])
            kapanislar.append(btc["Şimdi"][i])
            break

In [12]:
kapanislar=pd.DataFrame(kapanislar,columns=["kapanis"])
sonraki_kapanislar_=pd.DataFrame(sonraki_kapanislar_,columns=["yarin_kapanis"])

In [13]:
veriler=pd.concat([kapanislar,sonraki_kapanislar_,veriler],axis=1)

In [14]:
columns_titles = ["date","HABER","kapanis","yarin_kapanis","DURUM"]
veriler=veriler.reindex(columns=columns_titles)

In [15]:
veriler.iloc[:,0:4]

,date,HABER,kapanis,yarin_kapanis
0,20.06.2022,Investing.com - Kripto para piyasalarındaki gü...,20572.3,20720.400391
1,18.06.2022,Bitcoin 20.000 doların altına düştüInvesting.c...,18986.5,20577.199219
2,17.06.2022,Investing.com - Kripto para piyasalarındaki gü...,20444.6,18986.500000
3,16.06.2022,Investing.com - Kripto para piyasalarındaki gü...,20386.6,20444.599609
4,15.06.2022,Investing.com - Kripto para piyasalarındaki gü...,22577.9,20386.599609
...,...,...,...,...
1202,30.09.2018,Merkezi olmayan borsaya olan talep yüksektir. ...,6635.2,6608.200195
1203,28.09.2018,Investing.com – Kripto paralar Perşembe günü k...,6636.0,6603.899902
1204,27.09.2018,"Investing.com - Litecoin Perşembe günü %10,28 ...",6689.2,6636.000000
1205,26.09.2018,İngiltere menşeli The Knowledge Academy kurulu...,6468.1,6689.200195


In [16]:
for i in range(len(X)):
    X[i].append(veriler.kapanis[i])

In [17]:
y=veriler.yarin_kapanis

In [18]:
X = np.asarray(X).astype(np.float32)

In [19]:
maes=[]
import random
for i in range(1000):
    a=random.randint(0,10000)
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=a)
    from sklearn.ensemble import GradientBoostingRegressor
    gb = GradientBoostingRegressor(n_estimators=100) 
    gb.fit(X_train,y_train)
    predict=gb.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,predict)
    from sklearn.metrics import mean_absolute_error
    mae=mean_absolute_error(y_test,predict)
    pre_durum=[]
    for i in range(len(y_test)):
        if X_test[i][-1]<predict[i]:
            pre_durum.append("artar")
        else :
            pre_durum.append("azalir")
    real_durum=[]
    for j in range(len(y_test)):
        if X_test[j][-1]<list(y_test)[j]:
            real_durum.append("artar")
        else :
            real_durum.append("azalir")
    toplam=0
    for i in range(len(y_test)):
        if pre_durum[i]==real_durum[i]:
            toplam=toplam+1
    acc=toplam/len(y_test)
    maes.append(mae)

In [28]:
min(maes)

477.5169383362341

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=3664)
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=100) 
gb.fit(X_train,y_train)
predict=gb.predict(X_test)
from sklearn.metrics import r2_score
r2=r2_score(y_test,predict)
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test,predict)
print(mae)

475.86533683541865


In [22]:
real=pd.DataFrame(list(y_test),columns=["real"])
predict_df=pd.DataFrame(predict,columns=["predict"])

In [24]:
karsilastir=pd.concat([real,predict_df],axis=1)

In [26]:
karsilastir[0:20]

,real,predict
0,9655.700195,9692.226904
1,30097.800781,30307.151338
2,10672.900391,10259.394078
3,17730.699219,17255.388319
4,9631.200195,9672.228010
5,14019.900391,13424.307181
6,4004.100098,3912.835670
7,6468.899902,5726.989911
8,8000.399902,7975.701550
9,3990.399902,3984.535904
